In [ ]:
import pandas as pd
import os
from Bio import Entrez, SeqIO
import tqdm
from collections import Counter
import numpy as np
from scipy.stats import multinomial
import numpy as np
import ast


In [ ]:
iterator = pd.read_csv("/Users/reem/Mov/final_results_LLRs.tsv",sep="\t", chunksize=1000)
df = pd.concat([chunk for chunk in tqdm.tqdm(iterator, desc='Loading data')])

In [ ]:
def generate_all_possible_contexts(sub):
    bases = ["A","C","G","T"]
    possible_contexts = []
    for a in bases:
        for b in bases:
            possible_contexts.append(f"{a}[{sub}]{b}")
    return possible_contexts
print(generate_all_possible_contexts("G>A"))

In [ ]:
# Compute mean context probabilities
def get_mean_context_probs(df, llr_condition, prop_col, llr_col='LLR'):
    """
    Filters dataframe into llr>6 and llr<6 and computes mean 
    proportions per substitution per context

    Returns a flattened dataframe with mutational contexts,seqname,LLR
    and computed means as the last row

    """

    df_filtered = df[llr_condition(df[llr_col])].copy()
    
    pivot = pd.json_normalize(df_filtered[prop_col]).fillna(0)
    pivot = pivot.loc[(pivot != 0).any(axis=1)]

    return pivot.mean()

print(get_mean_context_probs(df, llr_condition=lambda x: x > 1, prop_col="T>Cproportions", llr_col='LLR'))
def build_prob_table(df,prop_col,llr_high,llr_low,llr_col='LLR'):
    """
    Builds a table of mean context probabilities (Molnupiravir vs Normal)
    for a specific substitution type.
    """
    llr_high=6
    llr_low=6

    Mov_probs = get_mean_context_probs(df.loc[(df!=0).any(axis=1)], lambda x: x > 6, prop_col='G>Aproportions')
    Normal_probs  = get_mean_context_probs(df.loc[(df!=0).any(axis=1)], lambda x: x < 6, prop_col='G>Aproportions')
    all_contexts = generate_all_possible_contexts("G>A")

    # Match means to all possible contexts
    Mov_probs = Mov_probs.reindex(all_contexts, fill_value=0)
    Normal_probs = Normal_probs.reindex(all_contexts, fill_value=0)

    print(Mov_probs)
    print(Normal_probs)

    df_prob = pd.DataFrame({'Mutational_Context':all_contexts,
                           'Molnupiravir': Mov_probs.values,
                            'Normal': Normal_probs.values
                            }) 

    
    return df_prob


df_prob = build_prob_table(df,prop_col='G>Aproportions',llr_high=6, llr_low=6, llr_col='LLR')

print(df_prob)

In [ ]:
print(df_prob)
print(df_prob["Molnupiravir"].sum())
print(df_prob["Normal"].sum())

In [ ]:
def calculate_llr(count_dict, pM, pN, contexts):
    counts = np.array([count_dict.get(ctx, 0) for ctx in contexts])
    n = counts.sum()
    if n == 0:
        return np.nan
    llM = multinomial.logpmf(counts, n=n, p=pM)
    llN = multinomial.logpmf(counts, n=n, p=pN)
    return float(llM - llN)

contexts = df_prob["Mutational_Context"].values.tolist()
pM = df_prob["Molnupiravir"].values.tolist()
pN = df_prob["Normal"].values.tolist()
df[f"G>A_llr"] = df["G>A_counts"].apply(lambda x: calculate_llr(x, pM, pN, contexts))

print(df[f"G>A_llr"])


In [ ]:
len(df)

In [ ]:
df.to_csv("/Users/reem/Mov/final_llrs_per_context.tsv", sep="\t", index=False) 

In [ ]:
"THE END"